# Inferentia2 EC2 에 Gradio UI Hugging Face TGI (Text Generation Inference) endpoint 생성
- 이 노트북은 [AWS-Neuron-llama-3-Korean-Bllossom-8B](AWS-Neuron-llama-3-Korean-Bllossom-8B) 모델을 inferentia2 의 TGI Docker 로 서빙하고 있는 포트(8080) 에 대한 Gradio Chat Interface 를 생성 합니다.
    - 참고: [Gradio App](https://www.gradio.app/)

---


## Gradio 설치

In [1]:
! pip install gradio

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 148.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 166.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 165.1 MB/s eta 0:00:00
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 175.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 129.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.20
    Uninstalling urllib3-1.26.20:
      Successfully uninstalled urllib3-1.26.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.37.38 requires urllib3<1.27,>=1.25.4; python_version < "3

## 사전 과제
- 이 노트북을 실행하는 Inferentia2 EC2의 8080 포트에 에 아래와 같이 TGI Docker 가 실행 중이어야 합니다. 
- ![ready_for_inference.png)](../img/ready_for_inference.png)

## InferenceClient 생성
- http://127.0.0.1:8080 로컬의 8080 포트에 연결하는 클라이언트를 생성 합니다.

In [2]:
import gradio as gr
from huggingface_hub import InferenceClient

client = InferenceClient(model="http://127.0.0.1:8080")

## 추론 함수 정의

In [3]:
def inference(message, history):
    partial_message = ""
    for token in client.text_generation(message, max_new_tokens=512, stream=True):
        partial_message += token
        yield partial_message

## Chat 인터페이스 생성
- server_name="0.0.0.0", server_port=8081 로 챗 인터페이스 생성
- [AWS-Neuron-llama-3-Korean-Bllossom-8B](AWS-Neuron-llama-3-Korean-Bllossom-8B) 모델을 inferentia2 의 TGI Docker 로 서빙하기

In [4]:
gr.ChatInterface(
    inference,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Chat with me!", container=False, scale=7),
    description="This is the demo for Gradio UI consuming TGI endpoint with  AWS-Neuron-llama-3-Korean-Bllossom-8B model, https://huggingface.co/Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B",
    title="Chat with AWS-Neuron-llama-3-Korean-Bllossom-8B model on Inferentia2, inf2.xlarge",
    examples=["1/ 딥러닝에 대해서 알려 주세요. 2/ Are tomatoes vegetables?"],
    retry_btn="Retry",
    undo_btn="Undo",
    clear_btn="Clear",
).queue().launch(server_name="0.0.0.0", server_port=8081, share=False)

Running on local URL:  http://0.0.0.0:8081

To create a public link, set `share=True` in `launch()`.
